In [1]:
#import all the libraries we need
from bs4 import BeautifulSoup
import requests
import MySQLdb as mdb
import datetime
from datetime import date, datetime, timedelta
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

import time
import arrow
import re
import requests
import json
import MySQLdb as mdb
import pandas as pd
from tabulate import tabulate
import matplotlib
import matplotlib.pyplot as plt
from slackclient import SlackClient

# Make the graphs a bit prettier, and bigger
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
plt.rcParams['figure.figsize'] = (15, 5)

In [ ]:
#!sudo -H pip3 install -U sqlalchemy

In [2]:
#this function takes the argument "parse" which is how many entries you want to read

def realm_grabber(parse):
    masterlist = [] #eventually this list will be returned
    #this for loop will parse the html for each page of recent deaths
    for i,j in enumerate(range(1,parse+1,100)):
        url = "https://www.realmeye.com/recent-deaths/{}?ms=1".format(j)
        print(url)
        page = requests.get(url)
        bs = BeautifulSoup(page.text, 'html.parser')
        total_chars = bs.findAll('tr')
        
        #this loops grabs all relevent character info stored in the html tag "td"
        for x in range(1, 101):
            char_data = {}
            num = j + x - 1 #this is going to be the primary key
            charlist = total_chars[x].find_all("td")
            
            #if the characters has a private profile, this character data will be added to the master list
            if charlist[1].contents[0] == "Private":
                char_data = {'char_id': num,
                            'char_base': None,
                            'char_name': 'hidden',
                            'char_oof': None,
                            'char_oofer': 'hidden',
                            'char_stats':'hidden',
                            'char_total': None,
                            'equips': {'Ability': 'hidden',
                                    'Armor': 'hidden',
                                    'Ring': 'hidden',
                                    'Weapon': 'hidden',
                                    'Back': 'hidden'}
                            }
                masterlist.append(char_data)
            #if the character has a public profile, this characters info will be organized below
            else:
            #creates a primary key for the character
                char_data["char_id"] = num
        
            #grabs the character's name
                char_data["char_name"] = charlist[1].find("a").contents[0]

            #grabs the cause of death
                char_data["char_oofer"] = charlist[7].contents[0]

            #grabs the time of death
                raw_time = charlist[2].contents[0]
                atime = raw_time.replace("T"," ")
                btime = atime.replace("Z","")
                char_data["char_oof"]= datetime.strptime(btime, '%Y-%m-%d %H:%M:%S').date()

            #grabs the character's stats
                char_data["char_stats"] = charlist[6].span.contents[0]
    
            #grabs the base fame
                char_data["char_base"] = int(charlist[3].contents[0])

            #grabs the total fame
                char_data["char_total"] = int(charlist[4].span.contents[0])

            #grabs the person's equips
                equiplist = ["Weapon","Ability","Armor","Ring","Back"]
                equips = {}
                i = 0
                for element in charlist[5]:
                    equips[equiplist[i]] = element.span["title"]
                    i += 1
                if i <= 5:
                    equips["Back"] = "Empty Slot"
            
                char_data["equips"] = equips
            
                masterlist.append(char_data)
                
    return masterlist

print("nice")

nice


In [3]:
masterlist = realm_grabber(1000) #uses the realm_grabber function to read the first 1000 entries off of realmeye 
#the number passed must be a multiple of 100

https://www.realmeye.com/recent-deaths/1?ms=1
https://www.realmeye.com/recent-deaths/101?ms=1
https://www.realmeye.com/recent-deaths/201?ms=1
https://www.realmeye.com/recent-deaths/301?ms=1
https://www.realmeye.com/recent-deaths/401?ms=1
https://www.realmeye.com/recent-deaths/501?ms=1
https://www.realmeye.com/recent-deaths/601?ms=1
https://www.realmeye.com/recent-deaths/701?ms=1
https://www.realmeye.com/recent-deaths/801?ms=1
https://www.realmeye.com/recent-deaths/901?ms=1


In [4]:
len(masterlist) #as you can see we have 1000 entries now in our masterlist

1000

In [5]:
#connect to mysql
con = mdb.connect(host = 'localhost', 
                  user = 'root', 
                  passwd = 'dwdstudent2015', 
                  charset='utf8', use_unicode=True);

# Query to create a database realmdata
db_name = 'realmdata'
drop_db_query = "DROP DATABASE IF EXISTS {db}".format(db=db_name)
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
cursor = con.cursor()
cursor.execute(drop_db_query)
cursor.execute(create_db_query)
cursor.close()

In [6]:
cursor = con.cursor()
table_name = 'recent_deaths'
# creates a table
# The {db} and {table} are placeholders for the parameters in the format(....) statement
drop_table_query= '''DROP TABLE IF EXISTS {db}.{table}'''.format(db = db_name, table= table_name)
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table}
                                (char_id int,
                                char_name varchar(250), 
                                death_cause varchar(250), 
                                date datetime,
                                char_stats varchar(250),
                                base_fame int,
                                total_fame int,
                                weapon varchar(250),
                                ability varchar(250),
                                armor varchar(250),
                                ring varchar(250),
                                back varchar(250),
                                PRIMARY KEY(char_id)
                                )'''.format(db=db_name, table=table_name)
cursor.execute(drop_table_query)
cursor.execute(create_table_query)
cursor.close()

print("nice")

nice


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: Warning: (1051, "Unknown table 'realmdata.recent_deaths'")


In [7]:

query_template = '''INSERT INTO {db}.{table}(char_id,
                                            char_name, 
                                            death_cause, 
                                            date,
                                            char_stats,
                                            base_fame,
                                            total_fame,
                                            weapon,
                                            ability,
                                            armor,
                                            ring,
                                            back) 
                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''.format(db=db_name, table=table_name)
cursor = con.cursor()

for entry in masterlist:
    char_id = entry['char_id']
    char_name = entry['char_name']
    death_cause = entry['char_oofer']
    date =  entry['char_oof']
    char_stats = entry['char_stats']
    base_fame = entry['char_base']
    total_fame = entry['char_total']
    weapon = entry["equips"]["Weapon"]
    ability = entry["equips"]["Ability"]
    armor = entry["equips"]["Armor"]
    ring = entry["equips"]["Ring"]
    back = entry["equips"]["Back"]
    
    query_parameters = (char_id, char_name, death_cause, date, char_stats, base_fame, total_fame,
                       weapon, ability, armor, ring, back)
    cursor.execute(query_template, query_parameters)
print("nice")

con.commit()
cursor.close()

nice


In [8]:
#connects to MySQL to pull info from our database

conn_string_realmdata = 'mysql://{user}:{password}@{host}:{port}/{db}'.format(
    user='root', 
    password='dwdstudent2015', 
    host = 'localhost', 
    port=3306, 
    db='realmdata'
)
engine_realmdata = create_engine(conn_string_realmdata)

In [9]:
query = '''
SELECT * FROM recent_deaths LIMIT 20
'''
total_realmdata = pd.read_sql(query, con=engine_realmdata)
total_realmdata

#our data from SQL can be pulled with a query

,char_id,char_name,death_cause,date,char_stats,base_fame,total_fame,weapon,ability,armor,ring,back
0,1,Olimar,Hot Lava,2017-12-12,7/8,1245.0,1601.0,Staff of the Cosmic Whole T12,Elemental Detonation Spell T6,Soulless Robe UT,Ring of the Pyramid UT,Empty Slot
1,2,hidden,hidden,NaT,hidden,NaN,NaN,hidden,hidden,hidden,hidden,hidden
2,3,BlackVault,Suffocation,2017-12-12,4/8,499.0,813.0,Dagger of Foul Malevolence T12,Cloak of the Planewalker UT,Griffon Hide Armor T12,Ring of Paramount Health T4,Empty Slot
3,4,Bobbykill,Oryx Insect Minion,2017-12-12,6/8,836.0,1399.0,Frosty's Walking Stick UT,Skullish Remains of Esben UT,Soulless Robe UT,Ring of Decades UT,Empty Slot
4,5,Antonioo,shtrs Defense System,2017-12-12,3/8,441.0,575.0,Frostbite UT,Mithril Shield T5,Abyssal Armor T12,Ring of Paramount Health T4,Empty Slot
5,6,XCrazyGuyL,Ice Tomb Attacker,2017-12-12,5/8,334.0,531.0,Saint Nicolas' Blade UT,Advent Seal T6,Abyssal Armor T12,Ring of the Northern Light UT,Empty Slot
6,7,Shimmers,Ice Tomb Attacker,2017-12-12,5/8,837.0,1385.0,An Icicle UT,Felwasp Toxin T4,Griffon Hide Armor T12,Amulet of Drakefyre UT,Empty Slot
7,8,hidden,hidden,NaT,hidden,NaN,NaN,hidden,hidden,hidden,hidden,hidden
8,9,hidden,hidden,NaT,hidden,NaN,NaN,hidden,hidden,hidden,hidden,hidden
9,10,Btdsts,Lava,2017-12-12,2/8,260.0,323.0,An Icicle UT,Nightwing Venom T5,Griffon Hide Armor T12,Ring of Paramount Magic T4,Empty Slot
